In [1]:
import pennylane as qml
import pennylane.numpy as np

In [376]:
def W(params):
    
    """
    Subcircuit that implements the trainable block W
    
    Args:
        params (np.array): A matrix containing the parameters for the trainable block W. The length of
        params is equal to the depth of the circuit. The length of each row in params is the number 
        of qubits used. See the challenge statement for a detailed explanation
    Returns:
        Since this function is a subcircuit, you must not return anything.
    
    """
    num_layers,num_wires=params.shape[0],params.shape[1]
    for i in range(num_layers):
        for j in range(num_wires):
            qml.RY(params[i,j], wires=j)
        # Apply CZ gate between each adjacent pair of wires
        for wire in range(num_wires-1 ):
            qml.CNOT(wires=[wire, wire + 1])
        qml.CNOT(wires=[num_wires-1,0])
        

In [349]:
def S(g, x, num_wires):
    """
    Subcircuit that implements the encoding block S
    
    Args:
        g (pennylane.Operator): A PennyLane operator representing the generator for the encoding
        gates. It must be Hermitian in order to generate a unitary. Call it as g(wires) to specify
        the wires on which it acts.
        x (complex): The scalar coefficient of the operator g.
        num_wires (int): The number of wires over which the encoding gate is broadcast.
        
    Returns:
        Since this function is a subcircuit, you must not return anything.
    """

    for wire in range(num_wires):
        qml.evolve(g(wire),coeff=-x)

In [350]:
num_wires=4
dev = qml.device("default.qubit", wires=num_wires)

In [377]:
@qml.qnode(dev)
def quantum_model(param_set, g, x):
    """
    This QNode implements the quantum model with alternating trainable and encoding blocks
    
    Args:
        param_set (np.array): A numpy array whose elements are the parameter matrices for each of the trainable
        blocks. Therefore, the length of this list is equal to the number of trainable blocks, which is greater
        than, or equal to 2.
        g (pennylane.Operator): A PennyLane operator representing the generator for the encoding
        gates. It must be Hermitian in order to generate a unitary.
        x: The scalar coefficient of the operator g.
    Returns:
        (np.tensor(float)): A tensor of dimensions (2,) representing the measurement probabilities in the computational 
        basis on the first wire.
    """
   

    for i in range(len(param_set)):
        mat = np.array(param_set[i])
        W(mat)
        if i == len(param_set)-1:
            break
        S(g,x,num_wires)  
    return qml.probs(wires=0)


In [379]:
g = qml.PauliY
x = 0.5236
param_set =np.array([[[0.62832, 0.3927, 1.0472, 0.7854],
                    [0.7854, 0.31416, 0.62832, 0.5236]],
                    [[0.31416, 0.7854, 0.7854, 0.3927],
                    [0.31416, 0.3927, 0.31416, 0.3927]]])
output = quantum_model(param_set, g, x)
print(output)


[0.68594115 0.31405885]


C:\Users\LENOVO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pennylane\math\utils.py:227: UserWarning: Contains tensors of types {'abc', 'autograd'}; dispatch will prioritize TensorFlow, PyTorch, and  Jax over Autograd. Consider replacing Autograd with vanilla NumPy.
  warnings.warn(


In [374]:
drawer= qml.draw(quantum_model)(param_set,g,x)
print(drawer)

0: ──RX(1.05)─╭●───────╭X──RX(1.05)─╭●───────╭X──Exp(0.79j X)──RX(1.05)─╭●───────╭X──RX(0.79)─╭●───
1: ──RX(0.79)─╰X─╭●────│───RX(0.79)─╰X─╭●────│───Exp(0.79j X)──RX(0.79)─╰X─╭●────│───RX(0.79)─╰X─╭●
2: ──RX(3.14)────╰X─╭●─│───RX(3.14)────╰X─╭●─│───Exp(0.79j X)──RX(1.57)────╰X─╭●─│───RX(1.57)────╰X
3: ──RX(0.39)───────╰X─╰●──RX(0.52)───────╰X─╰●──Exp(0.79j X)──RX(0.39)───────╰X─╰●──RX(0.79)──────

─────╭X─┤  Probs
─────│──┤       
──╭●─│──┤       
──╰X─╰●─┤       
